In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
#Nomes para as colunas
columns = ["Estacao","Data","Hora","Prec","TempMax","TempMin","Insolacao","EvapoPiche","TempCompMedia",
           "UmidRelatMedia", "VelocVento", "Missing"]

#Tipos de dados em cada coluna
types = {"Hora":np.uint16,"Prec":np.float64,"TempMax":np.float64,"TempMin":np.float64,"Insolacao":np.float64,
         "EvapoPiche":np.float64,"TempCompMedia":np.float64,"UmidRelatMedia":np.float64,"VelocVento":np.float64}

#Leitura do arquivo csv
dataSet = pd.read_csv("Dados_INMET/Diarios/INMET-Dados_Diarios_Natal_1968-2018.csv",sep=';',skiprows=48, nrows = 27003,
          usecols = columns[1:11], names=columns, dtype = types)

In [ ]:
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [ ]:
#Verificando dados
dataSet.head()

In [ ]:
#Verificando o tipo dos dados
dataSet.dtypes

In [ ]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]

In [ ]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSet00.head()

In [ ]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSet12.head()

In [ ]:
#Transformando a data no índice de cada dataframe
dataSet00 = dataSet00.set_index("Data")
dataSet12 = dataSet12.set_index("Data")

In [ ]:
#Descartando a coluna "Hora"
dataSet00, dataSet12 = dataSet00.drop(columns = "Hora"), dataSet12.drop(columns = "Hora")

In [ ]:
#Juntando os dados em um mesmo dataframe (no caso, o dataSet00)
for i in dataSet00.index:
    try:
        dataSet00["Prec"].loc[i] = dataSet12["Prec"].loc[i]
        dataSet00["TempMin"].loc[i] = dataSet12["TempMin"].loc[i]
    except:
        print("Data " + str(i.day) + "/" + str(i.month) + "/" + str(i.year) + " Não encontrada!")

In [ ]:
#Atribuindo à variável dataSet o DataFrame atualizado
dataSet = dataSet00
#Verificando dimensões do DataFrame
print(dataSet.shape)

In [ ]:
#Eliminando dias que estejam faltando variáveis
dataSet = dataSet.dropna()

In [ ]:
#Verificando dimensões do DataFrame após excluir linhas com problema
print(dataSet.shape)

In [ ]:
#Transformando a coluna Data de volta em uma coluna de informações do dataSet
dataSet = dataSet.reset_index()

# Considerando os seguintes intervalos: 
- Outono: 20 de março a 21 de junho (fechado em 20/03 e aberto em 21/06)
- Inverno: 21 de junho a 22 de setembro (fechado em 21/06 e aberto em 22/09)
- Primavera: de 22 de setembro a 21 de dezembro (fechado em 22/09 e aberto em 21/12)
- Verão: de 21 de dezembro a 20 de março (fechado em 21/12 e aberto em 20/03)

In [ ]:
#Função que retorna a estação do ano para uma determinada data
def season(data):
    if (data.month > 3 and data.month < 6) or (data.month == 3 and data.day >= 20) or (data.month == 6 and data.day < 21):
        return "Out"
    if (data.month > 6 and data.month < 9) or (data.month == 6 and data.day >= 21) or (data.month == 9 and data.day < 22):
        return "Inv"
    if (data.month > 9 and data.month < 12) or (data.month == 9 and data.day >= 22) or (data.month == 12 and data.day < 21):
        return "Pri"
    else:
        return "Ver"

In [ ]:
#Identificando a estação do ano na base de dados
estacao = []
for i in dataSet.index:
    estacao.insert(i, season(dataSet.loc[i, "Data"]))

In [ ]:
#Contando quantos dados existem para cada estação
print("Outono: " + str(estacao.count("Out")))
print("Inverno: " + str(estacao.count("Inv")))
print("Primavera: " + str(estacao.count("Pri")))
print("Verão: " + str(estacao.count("Ver")))

In [ ]:
#Adicionando a estação do ano na base de dados
dataSet["Season"] = estacao

In [ ]:
#Eliminando a coluna "Data" e visualizando dados
dataSet = dataSet.drop(columns="Data")
dataSet.head()

In [ ]:
#Separando a variável que será prevista (Y) em função das demais variáveis (X) e convertendo-os em um ndarray
X = np.array(dataSet.drop(columns = "Season"))
Y = np.array(dataSet["Season"])

In [ ]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 1)

In [ ]:
#Visualizando o conjunto de treino
print(X_train)
print(X_train.shape)

In [ ]:
#Efetuando uma transformação robusta nos dados
sc = RobustScaler(quantile_range = (5.0,95.0))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
classifier = SVC(C = 10000000.0, kernel = 'rbf', random_state = 0, gamma = 0.0001)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

In [ ]:
#Matriz de confusao, cujos valores da diagonal principal sao os acertos, acima da diagonal principal falsos-positivos e abaixo, falso-negativos
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Soma de todas as iteracoes feitas exibidas na matriz de confusao
total_iteracoes = 0
acerto_local = 0
for l in range(len(cm)):
    for c in range(len(cm[0])):
        total_iteracoes = total_iteracoes + cm[l][c]
        if l == c:
            acerto_local = acerto_local + cm[l][l]

In [ ]:
print(acerto_local / total_iteracoes)